In [12]:
import pandas as pd
import numpy as np
import time
import datetime
from IPython.display import clear_output
from collections import Counter
from functools import reduce
import itertools
from pbpstats.client import Client
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

In [2]:
settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "G:/My Drive/Sra_Coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
season = client.Season("nba", "2020-2021", "Regular Season")
# season = client.Season("wnba", "2021", "Regular Season")
games_id = []
k = 0
for final_game in season.games.final_games:
    k += 1
    clear_output(wait=True)
    print(k)
    games_id.append(final_game['game_id'])

1080


In [17]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "G:/My Drive/Sra_Coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
k = 0
for gameid in tqdm(games_id):
    clear_output(wait=True)
    k += 1
    # print(k)
    try:
        games_list.append(client.Game(gameid))
    except:
        print(gameid)
        bad_games_list.append(gameid)
        continue

100%|██████████| 1080/1080 [00:55<00:00, 19.45it/s]


In [5]:
len(bad_games_list)

11

In [6]:
player_dict = get_players(league='NBA',from_year=2020, to_year=2020)
player_id = [player['pID'] for player in player_dict]
team_dict = get_teams(league='NBA') # Creating Team Dictionary
team_id = [x['TeamID'] for x in team_dict ]

In [7]:
game = games_list[0]
possession = game.possessions.items[2]
possession_event = possession.events
event = possession_event[0]

In [15]:
# Get Location Data
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
game_id, period, clock, seconds_remaining,poss_length = [],[],[],[],[]
team_id, player1_id= [],[]
locX, locY, distance, shot_value, shot_type = [],[],[],[],[]
is_made, is_putback, is_assisted, player2_id  = [],[],[],[]
len_diffp = 0
pos_store = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, FieldGoal):
                try: 
                    game_id.append(possession_event.game_id)
                    period.append(possession_event.period)
                    clock.append(possession_event.clock)
                    seconds_remaining.append(possession_event.seconds_remaining)
                    poss_length.append(possession_event.seconds_since_previous_event)
                    team_id.append(possession_event.team_id)
                    player1_id.append(possession_event.player1_id)
                    locX.append(possession_event.locX)
                    locY.append(possession_event.locY)
                    distance.append(possession_event.distance)
                    shot_value.append(possession_event.shot_value)
                    shot_type.append(possession_event.shot_type)
                    is_made.append(possession_event.is_made)
                    
                    is_assisted.append(possession_event.is_assisted)
                    if possession_event.is_assisted:
                        player2_id.append(possession_event.player2_id)
                    else:
                        player2_id.append(0)
                    ii += 1
                    # print("ii = {0}".format(ii))
                    is_putback.append(possession_event.is_putback)
                except:
                    is_putback.append(False)
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

100%|██████████| 1069/1069 [00:05<00:00, 186.52it/s]


In [ ]:
player1_name = []
for pID in player1_id:
    player1_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player1_name = list(itertools.chain(*player1_name))
player2_name = []
for pID in player2_id:
    if pID==0:
        player2_name.append(['None'])
    else:     
        player2_name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
player2_name = list(itertools.chain(*player2_name))
team_name = []
for tID in team_id:  
    team_name.append([team['Team'] for team in team_dict if team['TeamID'] == tID])
team_name = list(itertools.chain(*team_name))

In [33]:
len(team_name)

188859

In [29]:
data = pd.DataFrame({'game_id':game_id,'period':period,'clock':clock,
                    'seconds_remaining':seconds_remaining,'poss_length':poss_length,
                    'team_id':team_id,'team_name':team_name,'player_id':player1_id,
                    'player_name':player1_name,'locX': locX,'locY':locY, 'distance':distance,
                    'shot_value':shot_value,'shot_type':shot_type,'is_made':is_made,
                    'is_assisted':is_assisted,'player_ast_id':player2_id,'player_ast_name':player2_name,
                    'is_putback':is_putback})
# data = data.sort_values(by=['Assists'],ascending=False)
# data = data.reset_index(drop=True)
data.head()

,game_id,period,clock,seconds_remaining,poss_length,team_id,team_name,player_id,player_name,locX,locY,distance,shot_value,shot_type,is_made,is_assisted,player_ast_id,player_ast_name,is_putback
0,0022000001,1,11:22,682.0,16.0,1610612751,Brooklyn Nets,202681,Kyrie Irving,-2,220,22.0,2,LongMidRange,True,True,201142,Kevin Durant,False
1,0022000001,1,11:11,671.0,11.0,1610612744,Golden State Warriors,1630164,James Wiseman,9,-2,0.9,2,AtRim,True,True,203952,Andrew Wiggins,False
2,0022000001,1,10:49,649.0,22.0,1610612751,Brooklyn Nets,201142,Kevin Durant,-20,258,25.9,3,Arc3,True,True,203925,Joe Harris,False
3,0022000001,1,10:31,631.0,18.0,1610612744,Golden State Warriors,203952,Andrew Wiggins,235,46,23.9,3,Corner3,False,False,0,None,False
4,0022000001,1,10:23,623.0,2.0,1610612751,Brooklyn Nets,201142,Kevin Durant,48,14,5.0,2,ShortMidRange,True,False,0,None,False


In [32]:
data.to_csv('ShotLocationData\Shot_Loc_2020_2021.csv',index=False)